# Listeners must have arguments "event" and "metadata" with proper types

In [11]:
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.project.domain.events.ProjectEvents import ProjectCreatedEvent
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to


class Invalid:
  pass


class ProjectorInvalid0:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self):
    print('Created project')


class ProjectorInvalid1:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid2:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
    print('Created project')


class ProjectorInvalid3:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: Invalid, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid4:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
    print('Created project')


event_publisher = EventPublisher()
try:
  projector = ProjectorInvalid0()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid1()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid2()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid3()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid4()
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x788b22559910>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x788b213b7ef0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x788b22559910>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x788b213b7ef0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x788b22559910>> m

# Cannot register same listener twice

In [12]:
class Projector:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
    print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
  event_publisher = EventPublisher()
  projector = Projector()
  event_publisher.register(projector)
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event handler already registered


# Create projects

In [13]:
from morpheus.project.infrastructure.persistence.ProjectSummaryRepository import project_summary_repository
from morpheus.project.infrastructure.event_sourcing.ProjectEventStore import project_event_store
from morpheus.project.types.User import UserId
from morpheus.project.types.Project import ProjectId, Name, Description, Tags
from morpheus.project.application.write.ProjectCommandHandlers import CreateProjectCommand, CreateProjectCommandHandler

for i in range(10):
  create_project_command = CreateProjectCommand(
    project_id=ProjectId.new(),
    name=Name.from_str(f'Project {i}'),
    description=Description.from_str(f'Description {i}'),
    tags=Tags.from_list([f'tag{i}', f'tag{i + 1}']),
    created_by=UserId.new()
  )
  CreateProjectCommandHandler.handle(create_project_command)

event_envelopes = project_event_store.get_all_events_ordered_by_version()
print(f'Found {len(event_envelopes)} events')
for event_envelope in event_envelopes:
  print(f'"{event_envelope.get_event().get_event_name().to_str()}" occurred at {event_envelope.event.get_occurred_at().to_iso_with_timezone()}')

project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 45 events
"Project Created" occurred at 2024-03-21T20:34:22.248171+01:00
"Project Created" occurred at 2024-03-22T09:51:38.989208+01:00
"Project Created" occurred at 2024-03-22T09:51:39.004992+01:00
"Project Created" occurred at 2024-03-22T09:51:39.011077+01:00
"Project Created" occurred at 2024-03-22T09:51:39.016934+01:00
"Project Created" occurred at 2024-03-22T09:51:39.022913+01:00
"Project Created" occurred at 2024-03-22T09:51:39.028560+01:00
"Project Created" occurred at 2024-03-22T09:51:39.033948+01:00
"Project Created" occurred at 2024-03-22T09:51:39.039459+01:00
"Project Created" occurred at 2024-03-22T09:51:39.044837+01:00
"Project Created" occurred at 2024-03-22T09:51:39.050090+01:00
"Project Created" occurred at 2024-03-22T09:52:01.760170+01:00
"Project Created" occurred at 2024-03-22T09:53:48.926500+01:00
"Project Created" occurred at 2024-03-22T09:55:40.497669+01:00
"Project Created" occurred at 2024-03-22T09:56:22.803405+01:00
"Project Created" occurred at 2024-03-2

# Reproject project summaries

In [14]:
from morpheus.project.presentation.cli.ProjectionCliCommands import ReprojectProjectSummariesCliCommand

ReprojectProjectSummariesCliCommand.run()

Reprojecting project summaries
------------------------------

Found 45 events
Reset projector
Successfully reprojected project summaries


In [15]:
project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 34 project summaries
Project 86eb03e2-423d-4a27-8c56-e33687c20f6f: Test1
Project cf85101d-12ef-4091-8ef6-f2aae73ac2c5: Project 0
Project b8c16723-da22-4d89-9b40-d144dbeab577: Project 1
Project 8b08fb8c-2fc4-459a-b09e-a0667ba67ca0: Project 2
Project ba6106fa-f233-4dfd-b778-ca18f17d8472: Project 3
Project e0f30817-eb4b-4867-b5d2-56b5ce1c7e85: Project 4
Project 25d8aed0-22c0-44cc-8e00-e582a3b723aa: Project 5
Project bc350acd-2813-4b54-885a-bf6a61514d5d: Project 6
Project d0d4cc80-d1bc-459c-95cd-9fd34692e306: Project 7
Project 6436b729-0018-428a-b4a2-7c3607cf2b6c: Project 8
Project e1ff8a08-4718-4572-80ba-bc4e1954e0e9: Project 9
Project ebd75250-4dcf-4586-b7d3-1905b8a4d3f1: Project 1
Project ae582838-717e-484e-b106-d8a26d067adb: Project 1
Project 94a70f09-5003-432e-9805-3c63f77d0420: Project 1
Project 522ed574-f0a1-40f5-81ec-bd443acdae01: Project 0
Project e0b2737a-688e-43e2-85f5-7b29b70b3aed: Project 1
Project 611a358a-dea8-4271-b627-b6b8c19fa2b4: Project 2
Project 3cb3b402-67f2-482